In [1]:
%env CUDA_VISIBLE_DEVICES=1
! echo $CUDA_VISIBLE_DEVICES

env: CUDA_VISIBLE_DEVICES=1
1


In [1]:
! pip install termcolor

In [7]:
ft_model = AutoModelForCausalLM.from_pretrained("/data/.cache/output_main_baseline/")
ft_tokenizer = AutoTokenizer.from_pretrained("/data/.cache/output_main_baseline/")

In [2]:
from codebook_features import models, run_clm, evaluation
import numpy as np
import pickle
from tqdm import tqdm
from transformers import GPTNeoXForCausalLM, AutoTokenizer, pipeline, set_seed, AutoModelForCausalLM
from matplotlib import pyplot as plt
from termcolor import colored
import torch

def logits_to_pred(logits, k=5):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True, dim=-1)
    probs = sorted_logits.softmax(dim=-1)
    topk_preds = [tokenizer.convert_ids_to_tokens(e) for e in sorted_indices[:, -1, :k]]
    topk_preds = [tokenizer.convert_tokens_to_string([e]) for batch in topk_preds for e in batch]
    return  topk_preds, sorted_logits[:,-1,:k], probs[:,-1, :k]


In [3]:
device = torch.device("cuda")
model_name_or_path = "EleutherAI/pythia-70m-deduped"
# model_name_or_path = "EleutherAI/pythia-410m-deduped"
# pretrained_path = "/shared/pre_proj_comp_cb_wiki_pythia/"
# pretrained_path = "/data/outputs/2023-04-24/13-23-43/output_main"
# pretrained_path = "taufeeque/best-cb-model"
# model_name_or_path = "gpt2"
# pretrained_path = "/shared/comp_cb_8_preproj_attn/"
# pretrained_path = "/data/gpt2_cb/"
# pretrained_path = "/nas/ucb/k8/codebook/outputs/2023-04-24/14-05-06/output_main/checkpoint-10000/"
# pretrained_path = "/nas/ucb/k8/codebook/outputs/2023-04-24/13-23-43/output_main/"
pretrained_path = "/nas/ucb/k8/codebook/outputs/2023-04-25/00-10-18/output_main/"
# pretrained_path = "/shared/gpt2_on_bc/"
orig_cb_model = models.wrap_codebook(model_or_path=model_name_or_path, pretrained_path=pretrained_path)
hooked_kwargs = dict(center_unembed=False,center_writing_weights=False,fold_ln=False,refactor_factored_attn_matrices=False,device="cpu")
cb_model = models.convert_to_hooked_model(model_name_or_path, orig_cb_model, hooked_kwargs=hooked_kwargs)
cb_model = cb_model.eval()
orig_cb_model = orig_cb_model.eval()
cb_model.model.cfg.device = device
cb_model.to(cb_model.model.cfg.device)


# model = HookedTransformer.from_pretrained(
#     "gpt2-small",
#     center_unembed=True,
#     center_writing_weights=True,
#     fold_ln=True,
#     refactor_factored_attn_matrices=True,
# )

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer


HookedTransformerCodebookModel(
  (model): HookedTransformer(
    (embed): Embed()
    (hook_embed): HookPoint()
    (blocks): ModuleList(
      (0-5): 6 x TransformerBlock(
        (ln1): LayerNorm(
          (hook_scale): HookPoint()
          (hook_normalized): HookPoint()
        )
        (ln2): LayerNorm(
          (hook_scale): HookPoint()
          (hook_normalized): HookPoint()
        )
        (attn): Attention(
          (hook_k): HookPoint()
          (hook_q): HookPoint()
          (hook_v): HookPoint()
          (hook_z): HookPoint()
          (hook_attn_scores): HookPoint()
          (hook_pattern): HookPoint()
          (hook_result): HookPoint()
          (hook_rot_k): HookPoint()
          (hook_rot_q): HookPoint()
        )
        (mlp): MLPWrapper(
          (module_layer): MLP(
            (hook_pre): HookPoint()
            (hook_post): HookPoint()
          )
          (codebook_layer): CodebookLayer(
            (codebook): Embedding(10000, 512)
            (l

In [1]:
import transformer_lens
model_name_or_path = "EleutherAI/pythia-70m-deduped"
hooked_kwargs = dict(center_unembed=False,center_writing_weights=False,fold_ln=False,refactor_factored_attn_matrices=False,device="cpu")
model = transformer_lens.HookedTransformer.from_pretrained(
    model_name_or_path,
    **hooked_kwargs,
)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer


In [4]:
model = cb_model
tokenizer = model.tokenizer

In [76]:
model.reset_hook_kwargs()
model.blocks[3].mlp.codebook_layer.hook_kwargs["disable_codes"] = []
model.blocks[3].mlp.codebook_layer.hook_kwargs["disable_topk"] = 1000
model.blocks[2].mlp.codebook_layer.hook_kwargs["disable_topk"] = 1000
model.blocks[1].mlp.codebook_layer.hook_kwargs["disable_topk"] = 1000
model.blocks[0].mlp.codebook_layer.hook_kwargs["disable_topk"] = 1000
model.blocks[4].mlp.codebook_layer.hook_kwargs["disable_topk"] = 1000
model.blocks[5].mlp.codebook_layer.hook_kwargs["disable_topk"] = 1000

In [34]:
tkns_on = tokenizer("The album was released on", return_tensors="pt")["input_ids"]
tkns_in = tokenizer("The album was released in", return_tensors="pt")["input_ids"]
model.reset_hook_kwargs()
for layer in range(model.cfg.n_layers):
    model.reset_hook_kwargs()
    print('Layer:', layer)
    model.blocks[layer].mlp.codebook_layer.hook_kwargs["disable_topk"] = 100
    logits,cache = model.run_with_cache(tkns_on)
    print(logits_to_pred(logits,k=10))
#     logits,cache = model.run_with_cache(tkns_in)
#     print(logits_to_pred(logits))


Layer: 0
([' the', ' a', ' two', ' three', ' iTunes', ' several', ' an', ' vinyl', ' various', ' one'], tensor([[34.2664, 33.5036, 32.6907, 31.8951, 31.8765, 31.5582, 31.5441, 31.4488,
         31.3393, 31.0215]], device='cuda:0', grad_fn=<SliceBackward0>), tensor([[0.2348, 0.1095, 0.0486, 0.0219, 0.0215, 0.0156, 0.0154, 0.0140, 0.0126,
         0.0091]], device='cuda:0', grad_fn=<SliceBackward0>))
Layer: 1
([' the', ' October', ' November', ' September', ' February', ' June', ' vinyl', ' CD', ' May', ' December'], tensor([[34.1595, 33.9832, 33.9243, 33.8500, 33.7136, 33.5355, 33.4316, 33.3697,
         33.2480, 33.0825]], device='cuda:0', grad_fn=<SliceBackward0>), tensor([[0.0717, 0.0601, 0.0567, 0.0526, 0.0459, 0.0384, 0.0346, 0.0325, 0.0288,
         0.0244]], device='cuda:0', grad_fn=<SliceBackward0>))
Layer: 2
([' the', ' September', ' November', ' October', ' February', ' June', ' May', ' December', ' January', ' July'], tensor([[36.9356, 36.9020, 36.8736, 36.8564, 36.7005, 36.6

In [94]:
print(tkns_on)

tensor([[ 510, 5400,  369, 4439,  327]])


In [7]:
tkns_on = tokenizer("The album was released on", return_tensors="pt")["input_ids"]
tkns_in = tokenizer("The album was released in", return_tensors="pt")["input_ids"]

In [8]:
tkns_on = tokenizer("The album was released on", return_tensors="pt")["input_ids"]
tkns_in = tokenizer("The album was released in", return_tensors="pt")["input_ids"]

for pos in range(tkns_in.shape[1]):
    model.reset_hook_kwargs()
    model.blocks[0].mlp.codebook_layer.hook_kwargs["disable_topk"] = 100
    model.blocks[0].mlp.codebook_layer.hook_kwargs["disable_for_tkns"] = [pos]
    logits,cache = model.run_with_cache(tkns_on)
    print(logits_to_pred(logits,k=5))



([' October', ' the', ' September', ' November', ' June'], tensor([[37.6049, 37.5924, 37.4603, 37.4531, 36.9519]], device='cuda:0',
       grad_fn=<SliceBackward0>))
([' the', ' September', ' October', ' November', ' February'], tensor([[35.5344, 35.4105, 35.3600, 35.3600, 35.1117]], device='cuda:0',
       grad_fn=<SliceBackward0>))
([' October', ' the', ' September', ' November', ' June'], tensor([[36.6905, 36.6711, 36.4966, 36.4599, 35.8196]], device='cuda:0',
       grad_fn=<SliceBackward0>))
([' the', ' September', ' June', ' October', ' November'], tensor([[35.3374, 34.8017, 34.7194, 34.6972, 34.5690]], device='cuda:0',
       grad_fn=<SliceBackward0>))
([' the', ' a', ' two', ' vinyl', ' iTunes'], tensor([[38.0003, 37.1255, 36.1537, 35.9368, 35.6996]], device='cuda:0',
       grad_fn=<SliceBackward0>))


In [8]:
block_codes = [1093, 7596, 5253, 5252,  676, 6262,  536, 1073, 6522, 1878, 4431, 4620,
    4103, 6638, 6619, 4189, 7561, 6364, 3587, 1766, 5073, 2069,  397, 1032,
    4759, 8514, 3410, 7218, 5315, 6492, 5681,  850, 6786, 5309, 3945, 8036,
    6539, 4155,   87, 9187, 4302, 6380,  783, 9502, 1495, 7014, 7027, 9789,
    6606, 5319]

# for idx in range(len(block_codes)):
#     model.reset_hook_kwargs()
#     model.blocks[0].mlp.codebook_layer.hook_kwargs["disable_codes"] = [block_codes[idx]]
#     logits,cache = model.run_with_cache(tkns_on)
#     print(logits_to_pred(logits,k=5))
    
model.reset_hook_kwargs()
model.blocks[0].mlp.codebook_layer.hook_kwargs["disable_codes"] = block_codes
logits,cache = model.run_with_cache(tkns_on)
print(logits_to_pred(logits,k=10))

([' the', ' a', ' two', ' vinyl', ' V', ' iTunes', ' September', ' October', ' November', ' three'], tensor([[37.0833, 35.6310, 35.5347, 35.0823, 35.0006, 34.6529, 34.5665, 34.5431,
         34.5405, 34.4171]], device='cuda:0', grad_fn=<SliceBackward0>), tensor([[0.2541, 0.0595, 0.0540, 0.0344, 0.0317, 0.0224, 0.0205, 0.0200, 0.0200,
         0.0177]], device='cuda:0', grad_fn=<SliceBackward0>))


In [37]:
cache['blocks.0.mlp.module_layer.hook_post'].shape

torch.Size([1, 5, 2048])

In [38]:
cache.keys()

dict_keys(['hook_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_rot_q', 'blocks.0.attn.hook_rot_k', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.module_layer.hook_pre', 'blocks.0.mlp.module_layer.hook_post', 'blocks.0.mlp.codebook_layer.hook_codebook_ids', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_rot_q', 'blocks.1.attn.hook_rot_k', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.module_l

In [9]:
block_codes_v = model.blocks[0].mlp.codebook_layer.codebook(torch.tensor(block_codes,device=device))

In [24]:
import torch.nn.functional as F
norm_block_codes_v = F.normalize(block_codes_v)
cosine_sim = norm_block_codes_v @ norm_block_codes_v.t()
cosine_sim[range(50), range(50)] = 0

In [33]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                z=cosine_sim.cpu().detach().numpy(),
                colorscale='Viridis'
            ))

fig.update_layout(
    title='Cosine Similarity Matrix',
    xaxis_title='Row Index',
    yaxis_title='Col Index'
)

fig.show()


In [9]:
model.reset_hook_kwargs()
logits,cache = model.run_with_cache(tkns_on)
print(logits_to_pred(logits,k=5))


([' October', ' the', ' November', ' September', ' June'], tensor([[37.2821, 37.2593, 37.0749, 37.0738, 36.5577]], device='cuda:0',
       grad_fn=<SliceBackward0>))


In [90]:
tkns = tokenizer("The album was released on", return_tensors="pt")["input_ids"]
logits,cache = model.run_with_cache(tkns)
print(logits_to_pred(logits))

([' October', ' the', ' November', ' September', ' June'], tensor([[37.2821, 37.2593, 37.0749, 37.0738, 36.5577]], device='cuda:0',
       grad_fn=<SliceBackward0>))


In [13]:
cache["blocks.0.mlp.codebook_layer.hook_codebook_ids"][0,4][:50]

tensor([1093, 7596, 5253, 5252,  676, 6262,  536, 1073, 6522, 1878, 4431, 4620,
        4103, 6638, 6619, 4189, 7561, 6364, 3587, 1766, 5073, 2069,  397, 1032,
        4759, 8514, 3410, 7218, 5315, 6492, 5681,  850, 6786, 5309, 3945, 8036,
        6539, 4155,   87, 9187, 4302, 6380,  783, 9502, 1495, 7014, 7027, 9789,
        6606, 5319], device='cuda:0')

In [78]:
logits,cache = model.run_with_cache("The album was released in")
print(logits_to_pred(logits))

([' the', ' a', ' to', ' "', ' and'], tensor([[30.4220, 29.2685, 28.5096, 28.3555, 28.2532]], device='cuda:0',
       grad_fn=<SliceBackward0>))


In [ ]:
cache["blocks.3.mlp.codebook_layer.hook_codebook_ids"]

In [42]:
logits_to_pred(logits)

(['@', ' =', ' @', ' )', ' million'],
 tensor([[59.1928, 43.5733, 42.5622, 42.1242, 42.1061]], device='cuda:0',
        grad_fn=<SliceBackward0>))

In [36]:
model.model.generate("It was literally a 20 @-", max_new_tokens=50, do_sample=False, temperature=0.7)

  0%|          | 0/50 [00:00<?, ?it/s]

'It was literally a 20 @-@ year @-@ old nuclear reactor design with a tetraethyllead and a half @-@ hour @-@ old cladding. It was an enormous leap to attract a million tonnes of radioactive waste. \n The Manhattan Project was the first major'

In [ ]:
prefixes = ["The largest", "One of the most", "your time is limited", "Once upon a time,"]
# prefixes = ["\n = = The Story =", "\n = = Hello =", "\n = = The Hello2 =", "\n = = This is a very long title =", "\n = = Career ="]
best_cb_generator = pipeline('text-generation', model=orig_cb_model.model, tokenizer=cb_model.tokenizer, device=0)
for prefix in prefixes:
    for i in range(3):
        print(best_cb_generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

In [ ]:
prefixes = ["The largest", "One of the most", "your time is limited", "Once upon a time,"]
# prefixes = ["\n = = The Story =", "\n = = Hello =", "\n = = The Hello2 =", "\n = = This is a very long title =", "\n = = Career ="]
best_cb_generator = pipeline('text-generation', model=ft_model, tokenizer=ft_tokenizer, device=0)
for prefix in prefixes:
    for i in range(3):
        print(best_cb_generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

In [ ]:
! sudo -E /opt/conda/bin/python /data/codebook-features/eval.py \
    --model_name_or_path $model_name_or_path --pretrained_path $pretrained_path \
    --dataset_name bookcorpus --dataset_config_name plain_text

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
04/07/2023 07:12:14 - WARNING - codebook_features.run_clm - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
Dataset wikitext downloaded and prepared to /shared/huggingface/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.
Running tokenizer on dataset:  36%|▎| 647000/1801350 [01:31<02:30, 7663.74 examp[WARNING|tokenization_utils_base.py:3561] 2023-04-07 07:14:16,539 >> Token indices sequence length is longer than the specified maximum sequence length for this model (1059 > 1024). Running this sequence through the model will result in inde

In [ ]:
! ls /shared

2023-03-09_23-21-51  2023-04-07_07-12-14     gpt2_on_bc   output_main_baseline
2023-03-09_23-58-22  2023-04-10_08-11-48     huggingface  outputs
2023-03-10_00-07-02  comp_cb_8_preproj_attn  n.txt
2023-03-31_11-51-32  f.txt		     output_main


In [24]:
# base_dir = "2023-02-17_09-27-39/"
# base_dir = "2023-03-02_21-17-36/"
# base_dir = "2023-03-31_11-51-32/"
# base_dir = "2023-04-07_07-12-14/"
# base_dir = "/shared/2023-04-10_08-11-48/"
# base_dir = "/shared/2023-04-14_13-47-57/"
# base_dir = "/homedir/2023-04-25_15-00-13/" # mlp: https://wandb.ai/interpretability/gpt2-codebook/runs/v3d1bt77
# base_dir = "/home/taufeeque/codebook-features/2023-04-28_07-43-57/"
base_dir = "/home/taufeeque/codebook-features/2023-04-28_08-37-37/"
cb_at = "mlp"
ccb = ""
tokens = np.load(base_dir + "tokens.npy", allow_pickle=True)
# cb_acts_complete = np.load("2023-02-17_09-27-39/cb_acts.npy", allow_pickle=True).item()
with open(base_dir + "cb_acts.pkl","rb") as f:
    cb_acts_complete = pickle.load(f)
metrics = np.load(base_dir + "metrics.npy", allow_pickle=True)
num_codes = 10_000

In [25]:
samples = 10_000
tokens_trunc = tokens[:samples]
cb_acts = {}
for k, v in cb_acts_complete.items():
    cb_acts[k] = v[:samples]


In [7]:
print(tokens.shape)

(10000, 1024)


In [9]:
print(metrics)
print(list(cb_acts.keys()))
print(cb_acts[f"layer0_{cb_at}{ccb}"].shape)

{'eval_loss': 3.0772526264190674, 'eval_accuracy': 0.42245865102639296, 'eval_runtime': 311.6289, 'eval_samples_per_second': 32.089, 'eval_steps_per_second': 1.004, 'eval_multicode_k': 100, 'eval_dead_code_fraction/layer0': 1.0, 'eval_MSE/layer0': 0.0, 'eval_input_norm/layer0': 0.0, 'eval_output_norm/layer0': 0.0, 'eval_dead_code_fraction/layer1': 1.0, 'eval_MSE/layer1': 0.0, 'eval_input_norm/layer1': 0.0, 'eval_output_norm/layer1': 0.0, 'eval_dead_code_fraction/layer2': 1.0, 'eval_MSE/layer2': 0.0, 'eval_input_norm/layer2': 0.0, 'eval_output_norm/layer2': 0.0, 'eval_dead_code_fraction/layer3': 1.0, 'eval_MSE/layer3': 0.0, 'eval_input_norm/layer3': 0.0, 'eval_output_norm/layer3': 0.0, 'eval_dead_code_fraction/layer4': 1.0, 'eval_MSE/layer4': 0.0, 'eval_input_norm/layer4': 0.0, 'eval_output_norm/layer4': 0.0, 'eval_dead_code_fraction/layer5': 1.0, 'eval_MSE/layer5': 0.0, 'eval_input_norm/layer5': 0.0, 'eval_output_norm/layer5': 0.0}
['layer0_mlp', 'layer1_mlp', 'layer2_mlp', 'layer3_mlp

In [ ]:
k, v = np.unique(cb_acts["layer0_attn_ccb1"], return_counts=True)
plt.bar(k, v)
plt.yscale('log')
plt.show()

v_sortind = np.argsort(-v)
k, v = k[v_sortind], v[v_sortind]
plt.bar(range(len(v)), v)
plt.yscale('log')
plt.show()

In [ ]:
import plotly.express as px
num_tkns_for_each_code = [len(tkns) for tkns in ft_tkns if len(tkns) < 100]
px.histogram(num_tkns_for_each_code)

In [26]:

def features_to_tokens(cb_key,n=10):
    codebook_ids = cb_acts[cb_key]
    features_tokens = [[] for _ in range(num_codes)]
    for i in tqdm(range(codebook_ids.shape[0])):
        for j in range(codebook_ids.shape[1]):
            for k in range(codebook_ids.shape[2]):
                past_future_tuple = (tokens[i, max(0, j-n):j+1], tokens[i, j+1:j+n])
                features_tokens[codebook_ids[i,j,k]].append(past_future_tuple)

    return features_tokens

def features_to_tokens_fast(cb_key,n=10):
    codebook_ids = cb_acts[cb_key]
    features_tokens = [[] for _ in range(num_codes)]
    for i in tqdm(range(codebook_ids.shape[0])):
        for j in range(codebook_ids.shape[1]):
            for k in range(codebook_ids.shape[2]):
                features_tokens[codebook_ids[i,j,k]].append((i,j))

    return features_tokens


In [28]:
ft_tkns = features_to_tokens_fast(f"layer0_{cb_at}{ccb}")
# ft_tkns2 = features_to_tokens_fast(f"layer2_{cb_at}{ccb}")
# ft_tkns5 = features_to_tokens_fast(f"layer5_{cb_at}{ccb}")

100%|█████████████████████████████████████████████████████████████████████████████| 10000/10000 [06:19<00:00, 26.37it/s]


In [29]:
ft_tkn3 = features_to_tokens_fast(f"layer3_{cb_at}_ccb0")

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:30<00:00, 323.38it/s]


In [30]:
import numpy as np
def tkn_print(ll, n=3):
    indices = np.random.choice(len(ll), min(len(ll), 100), replace=False)
    for idx in indices:
        i, j = ll[idx]
        print(f"example - {i}, token - {j}")
        print(tokenizer.decode(tokens[i,max(0,j-n):j]), end='')
        print(colored(tokenizer.decode(tokens[i,j]), 'red'), end='')
        print(tokenizer.decode(tokens[i,j+1:j+n]))
    print(colored("************************************************************", "green"))
    

In [34]:
def print_ft_tkns(ft_tkns, n=3, start=0, stop=1000, indices=None):
    indices = list(range(start, stop)) if indices is None else indices
    for i in indices:
#         unique_tokens = set(ft_tkns[i])
#         if 0 < len(unique_tokens) < 100:
#             print(i, len(unique_tokens), len(ft_tkns[i]))
#             tkn_print(unique_tokens)
#             print("**********************************")
        tkns = ft_tkns[i]
        if len(tkns) > 0:
            print(f"code - {i}, mapped to {len(tkns)} tokens")
            tkn_print(tkns, n)

In [35]:
print_ft_tkns(ft_tkns,n=10,indices=block_codes)

code - 1093, mapped to 112727 tokens
example - 633, token - 142
 that a review of his personal correspondence during the war reveals a tendency for self @-@ aggrand
example - 2944, token - 772
 that not being able to find key items in dungeons was a drawback of the system. 
 Review
example - 6116, token - 592
 both the Senate and the Roman people, though both submitted through necessity. Elagabalus'reputation
example - 9222, token - 147
 cells. 
 The impact of sickle cell trait on malaria immunity illustrates some evolutionary trade @-@
example - 9970, token - 997
 whom the author suggests was a Southerner, made " little effort to disguise his racial prejudice during
example - 8367, token - 92
 maintain visibility. 
 At the OVW tapings on May 10, Kennedy returned to in @-
example - 4202, token - 781
 tops of the curtain walls allowed defenders to rain missiles on enemies below, and battlements gave them further
example - 5670, token - 858
 media coverage of the play was " unfortunately

 from Antwerp. Because the twins were born on a U.S.-flagged vessel,
example - 2320, token - 370
 oldest extant metal truss bridge. 
 The Capon Lake Whipple Truss Bridge was originally constructed
example - 6570, token - 720
 surrender. 
 Winnebago was placed in ordinary on the Algiers side of the Mississippi,
************************************************************
code - 6492, mapped to 114920 tokens
example - 945, token - 1011
ega of America subsidiary, which executed a limited launch on August 14, 1989, in New York City
example - 2431, token - 408
 and " Heroes ". Emeli Sandé features on " A Heart Can Save the World ",
example - 6246, token - 306
 ship as a newly @-@ born star shining on the land, similar to Venus in the evening
example - 3509, token - 86
 600 meters ( 5 @,@ 200 ft ) above the sea level. A study in southeastern
example - 6336, token - 303
 was reached a month later. 
 Further complications occurred on May 20, 2009, when executive producer Peter
example - 7165, 

In [20]:
print_ft_tkns(ft_tkns2, 10)

code - 332, mapped to 9220 tokens
example - 9079, token - 226
'mun al @-@ Bata <unk> during the caliphate of Imam Al
example - 8184, token - 280
 ( born 1981 ), and daughter, <unk> <unk> Soetoro ( born 1987
example - 6358, token - 286
 Tarcutta, Ladysmith, <unk>, <unk> and Uranquinty
example - 9393, token - 300
 ( known in Malay as the Johan <unk> Negara or <unk> ) by the
example - 3863, token - 225
's estranged wife. 
 Vincent <unk> as Michael Cerrito. 
 Victor Rivers as
example - 5629, token - 504
 concentration camps, primarily at <unk> <unk> in Belgrade. 
 = = History =
example - 4065, token - 633
 bin Fakhruddin Hasani <unk> in 848 AH 
 At the library of
example - 1379, token - 367
 her tent and sends for Mirri Maz <unk> ( Mia Soteriou ) to
example - 7521, token - 737
 questions the court threw at him ; he <unk> answered those that agreed with his stance, and
example - 486, token - 389
-@ 3. 
 ( Norwegian ) <unk>, Odd ( 1979 ). <unk>
example - 1862, token - 680
 Collins, Jr. as CC

In [34]:
print_ft_tkns(ft_tkn3, 10)

code - 68, mapped to 1 tokens
example - 6942, token - 311
-@ 62 surfaced in the midst of the group. The submarine launched its only remaining torpedo at
************************************************************
code - 87, mapped to 336 tokens
example - 107, token - 756
 only minor changes since then. 
 The section of US @-@ 64 between Webbers Falls and
example - 2186, token - 68
 and the combined road heads north into Fort Lee. US 1 / 9 / 46 come to an interchange
example - 3889, token - 627
 North of Harrietstown, the former alignment of NY 3 became NY 408 from Harrietstown to
example - 3889, token - 801
 in much the same way as its spur route as NY 86 was rerouted to follow NY 192
example - 9067, token - 550
 route remained unchanged until January 1, 1970, when NY 216 was truncated to its current eastern terminus in
example - 3188, token - 451
. Here, it intersects the eastern terminus of NY 366 at a junction situated adjacent to Fall Creek
example - 8254, token - 415
 NY 386 was 

In [23]:
print_ft_tkns(ft_tkns5,n=10)

code - 47, mapped to 1 tokens
0: example - 1736, token - 88
 feng shui ), the fox's power over evil is such that a mere statue of
************************************************************
code - 180, mapped to 122 tokens
0: example - 508, token - 480
 and published by Capcom. It is a spin @-@ off title in the original Mega Man
1: example - 508, token - 481
 published by Capcom. It is a spin @-@ off title in the original Mega Man series
2: example - 762, token - 399
 and five expansions on gaming consoles. There are spin @-@ offs for Windows and Macint
3: example - 762, token - 400
 five expansions on gaming consoles. There are spin @-@ offs for Windows and Macintosh
4: example - 762, token - 401
 expansions on gaming consoles. There are spin @-@ offs for Windows and Macintosh systems
5: example - 1774, token - 938
 in Japan on November 20, 1995. A spin @-@ off novel written by Endo Ak
6: example - 1774, token - 939
 Japan on November 20, 1995. A spin @-@ off novel written by Endo Ak

In [ ]:
import matplotlib.pyplot as plt

def token_to_dist_of_fts(token, layer_key):
    token_idx = tokenizer.convert_tokens_to_ids(token)
    print(token_idx)
    layer_cb_acts = cb_acts[layer_key]
    token_present = tokens == token_idx
    fts_of_token = layer_cb_acts[token_present]
    print(fts_of_token.shape)
    plt.hist(fts_of_token[:, :3], bins=100, label=["1", "2", "3"])
    plt.legend()
    plt.title(f'Features Queried | Layer = {layer_key}')
    plt.show()

token_to_dist_of_fts('ĠAfrica', 'layer0_attn_ccb0')
token_to_dist_of_fts('ĠAfrica', 'layer1_attn_ccb0')
token_to_dist_of_fts('ĠAfrica', 'layer2_attn_ccb0')
token_to_dist_of_fts('ĠAfrica', 'layer3_attn_ccb0')
token_to_dist_of_fts('ĠAfrica', 'layer4_attn_ccb0')
token_to_dist_of_fts('ĠAfrica', 'layer5_attn_ccb0')

In [2]:
model = models.wrap_codebook(model_or_path="EleutherAI/pythia-70m-deduped", pretrained_path="taufeeque/pythia-attn-cb")

In [137]:
prefixes = ["Hello I am", "What is the", "The largest", "One of the most", "Your time is limited", "Keep your friends close, but", "Once upon a time,"]
# prefixes = [""]

In [139]:
orig_generator = pipeline('text-generation', model=orig_model, tokenizer=tokenizer)
for prefix in prefixes:
    for i in range(3):
        print(orig_generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello I am a non-member of a team in the USA.

My question is: How do I make a roster in one team?

I am not a member of the Team in the USA. I am in the USA and


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello I am a newbie with a background in UI.
<xmpp> i'm using a simple text editor but i don't use the web page when it's a text editor
<xmpp> and i've already found


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Keep your friends close, but that's what you make of him."

"How do you know that?"

"I never knew you'd be here," she said. "I never expected you to tell me what you do, and


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Once upon a time, my daughter used to be a very active participant in social distancing.

I had a friend who was in a group together who knew his wife, and he was on the run for a bit for it. She was


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Once upon a time, the same is true of the former, at the time of the loss of the former and the present, of the latter.

The problem is with the loss of the former. For example, the loss of the former
Once upon a time, I just found that the
washers were not so simple as to be able to handle the entire
work, and it was not the time for me to simply do the same. I
had a few different questions about


In [140]:
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model=model.model, tokenizer=tokenizer)
for prefix in prefixes:
    for i in range(3):
        print(generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello I am a child to be a person of the family. " 
 = = = Reception = = = 
 " The episode of the episode " is a guest @-@ cast @-@ action thriller, which includes guest @-@


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello I amai's wife ". The first female of the American writer of the American columnist, and was the authorial, having been inspired by his earlier and @-@ fiction that his " character was " more popular in the United States


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Keep your friends close, but to which their families were to have the house. 
 = = = Other things that are used for the Lord of Lord of the Earl of England and his father of William H. HMS, they had been at the


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Once upon a time, the player is having a career ; " the game is a good and the guy. " 
 The game was released in the United States in the World War I, who gave his first role as the first @-@ game


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Once upon a time,, the fate of this was so many of the world's efforts. 
 = = = Geography of the world = = = = 
 The Great East, the first of the Northern United Nations and the World War (
Once upon a time,, these were not seen in " the episode ", but the episode was originally aired on The MHS and the Fox Show. The film premiered on the Fox Show at the Fox Show. The film was written by the film


In [2]:
best_cb_model = models.wrap_codebook(model_or_path="EleutherAI/pythia-70m-deduped", pretrained_path="taufeeque/pythia-100mc")

Some weights of the model checkpoint at taufeeque/pythia-100mc were not used when initializing GPTNeoXCodebookModel: ['model.gpt_neox.layers.2.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.1.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.5.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.0.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.3.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.4.attention.codebook_layer.codebook.0.counts']
- This IS expected if you are initializing GPTNeoXCodebookModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPTNeoXCodebookModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifica

In [13]:
from codebook_features import models
import numpy as np
import pickle
from tqdm import tqdm
from transformers import GPTNeoXForCausalLM, AutoTokenizer
from matplotlib import pyplot as plt
from termcolor import colored

tokenizer = AutoTokenizer.from_pretrained(
  "EleutherAI/pythia-70m-deduped",
)
orig_model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/pythia-70m-deduped")
model = models.wrap_codebook(model_or_path="EleutherAI/pythia-70m-deduped", pretrained_path="taufeeque/pythia-100mc")

vis = {}


def hook_fn(m, i, o):
    vis[m] = (i, o)

vis_attn = {}
    
def hook_fn_attn(m, i, o):
    vis_attn[m] = (i, o)

    
for name, layer in model.model.gpt_neox.layers._modules.items():
    print(name)
    layer.attention.codebook_layer.register_forward_hook(hook_fn)
    layer.attention.register_forward_hook(hook_fn_attn)


s = "This tokenizer has been trained to treat spaces like parts of the tokens random tokens added here"
inp = tokenizer(s, return_tensors="pt")
out = model(**inp)


Some weights of the model checkpoint at taufeeque/pythia-100mc were not used when initializing GPTNeoXCodebookModel: ['model.gpt_neox.layers.3.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.5.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.2.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.1.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.4.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.0.attention.codebook_layer.codebook.0.counts']
- This IS expected if you are initializing GPTNeoXCodebookModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPTNeoXCodebookModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifica

0
1
2
3
4
5


In [14]:
for i, layers in model.all_codebooks.items():
    for cb in layers:
        print(cb.reconstruction_mse)
        print('cb_norm',cb.codebook[0].avg_norm())
#         print(cb.codebook[0].input.norm())
#         print(cb.codebook[0].output.norm())

44.75636291503906
cb_norm 22.606342315673828
150.0145721435547
cb_norm 22.624954223632812
953.9581298828125
cb_norm 22.625259399414062
567.1958618164062
cb_norm 22.624855041503906
977.8589477539062
cb_norm 22.627485275268555
3250.580322265625
cb_norm 22.615549087524414


In [3]:
model.reset_codebook_metrics()

In [22]:
import importlib
importlib.reload(models)


<module 'codebook_features.models' from '/home/taufeeque/codebook-features/codebook_features/models.py'>

In [ ]:
24.59918189048767
58.6542329788208
1281.2109985351562
1219.846923828125
2715.999969482422
8037.982604980469


In [3]:
model = best_cb_model

In [24]:
print(len(vis))
print(vis.keys())


6
dict_keys([CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
), CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
), CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
), CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
), CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
), CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
)])


In [72]:
for module, (i, o) in vis.items():
    i, o = i[0], o
    print(i.norm(p=2,dim=-1).mean().item(), o.norm(p=2,dim=-1).mean().item())
    print((((i-o)).norm(p=2,dim=-1)**2).mean().item())

7.211728572845459 2.2341723442077637
44.75636291503906
13.70687484741211 2.742445230484009
150.0145721435547
32.99772262573242 3.2736246585845947
953.9581298828125
25.82971954345703 3.1037824153900146
567.1959228515625
33.30326843261719 3.3043413162231445
977.8590698242188
58.66970443725586 3.357837438583374
3250.580322265625


In [11]:
# model.disable_codebooks()
model.reset_codebook_metrics()
vis_attn = {}
out = model(**inp)

for module, (i, o) in vis_attn.items():
    i, o = i[0], o[0]
    print(o.norm(p=2,dim=-1).mean().item())

7.211728572845459
11.842400550842285
29.040924072265625
22.245229721069336
13.874805450439453
21.93659019470215


In [10]:
model.all_codebooks
# wrapper = list(vis_attn.keys())[0]
# print(wrapper.snap)
for layer in vis_attn:
    layer.snap = False

In [44]:
def get_norm(t):
    return t.norm(p=2,dim=-1).mean().item()

In [48]:
last_cb = model.all_codebooks[5][0].codebook[0]

last_o = last_cb(i)
print(get_norm(last_o))

3.357837438583374


In [53]:
import torch
logits = -torch.cdist(i, last_cb.codebook.weight, p=2)
_, codebook_ids = logits.topk(models.BaseSnapFunction.k, dim=-1)
outputs = torch.nn.functional.embedding(codebook_ids, last_cb.codebook.weight)

In [56]:
print(get_norm(outputs))
outputs_avg = outputs.sum(dim=-2) / models.BaseSnapFunction.k
print(get_norm(outputs_avg))

22.306095123291016
3.357837438583374


In [57]:
outputs.shape

torch.Size([1, 18, 100, 512])

In [58]:
outputs_avg.shape

torch.Size([1, 18, 512])

In [71]:
orig_vis = {}


def orig_hook_fn(m, i, o):
    orig_vis[m] = (i, o)

for name, layer in orig_model.gpt_neox.layers._modules.items():
    print(name)
    layer.attention.register_forward_hook(orig_hook_fn)

orig_out = orig_model(**inp)

0
1
2
3
4
5


In [70]:
for module, (i, o) in orig_vis.items():
    i, o = i[0], o[0]
    print(o.norm(p=2,dim=-1).mean().item())

3.2817928791046143
3.281010866165161
4.416781425476074
3.543278455734253
2.703380584716797
14.310317993164062


In [65]:
orig_vis.keys()

dict_keys([])

In [ ]:
import re
pattern = "\n\s*=[^=]*=\s*\n"
def color_pattern(s, p=pattern):
    """colors p in s using termcolor"""
    return re.sub(p, colored(r"\g<0>", "blue", attrs=['bold']), s)

# for idx in range(len(tokens)):
#     example = tokens[idx]
#     print(color_pattern(tokenizer.decode(example)))
#     if idx > 100:
#         break

for idx in range(len(tokens)):
    example = tokens[idx]
    s = tokenizer.decode(example)
    matches = re.findall(pattern, s)
    if len(matches) > 0:
        print(idx, matches)

In [23]:
generator = pipeline('text-generation', model=model.model, tokenizer=tokenizer)
prefix = "When Mary and John went to the store, John gave a drink to"
print(generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


When Mary and John went to the store, John gave a drink to a family member. 
 = = = The householder = = = 
 The house was built by the architect William Horton, in the early 17th century. When the house


In [19]:
orig_generator = pipeline('text-generation', model=orig_model, tokenizer=tokenizer)
print(orig_generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


When Mary and John went to the store, John gave a drink to Mary’s mother of the children and said, “We’re going to get you something nice for me and I’m going to get you something good for me and I
